In [10]:
path = '7x7\maps'
extension = '.npy'

import numpy as np
import os

path = '7x7\maps'
extension = '.npy'

# import the numpy array
def import_map(map_name):
    map_path = os.path.join(path, str(map_name) + extension)
    return np.load(map_path)

a = import_map(0)
np.set_printoptions(threshold=np.inf)

print(a.shape)

(350, 350)


In [12]:
b = np.zeros((2, 7, 7), dtype=int)
b

array([[[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]])